In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

# EDA 이미지 저장 폴더 생성
os.makedirs('eda_image', exist_ok=True)

# 그래프 스타일 설정
plt.style.use('default')
sns.set_palette("husl")

In [2]:
import pandas as pd

def load_dataset(path:str) -> pd.DataFrame:
    return pd.read_csv(path)


def split_features_targets(df_dataset:pd.DataFrame, target_name:str) -> tuple:
    y_df = df_dataset[target_name]
    x_df = df_dataset.drop(target_name, axis=1)
    return x_df, y_df


def do_load_dataset(train_path:str, test_path:str, target_name:str):
    df_train_full = load_dataset(path=train_path)
    df_test = load_dataset(path=test_path)

    x_tr, y_tr = split_features_targets(
        df_dataset=df_train_full, target_name=target_name)
    
    x_te, y_te = split_features_targets(
        df_dataset=df_test, target_name=target_name)
    
    return x_tr, x_te, y_tr, y_te

In [3]:
x_tr, x_te, y_tr, y_te = do_load_dataset(train_path="./data/hotel_bookings_train.csv", test_path="./data/hotel_bookings_test.csv", target_name="is_canceled")

In [4]:
'''
피처 생성
x_tr, x_te
'''

import pandas as pd
import numpy as np

def do_feature_extraction(df_train:pd.DataFrame, df_test:pd.DataFrame):
    # has_conpany
    df_train['has_company'] = (df_train['company'] > 0).astype(int)
    df_test['has_company'] = (df_test['company'] > 0).astype(int) 

    # has_agent
    df_train['has_agent'] = (df_train['agent'] > 0).astype(int)
    df_test['has_agent'] = (df_test['agent'] > 0).astype(int)

    # is_FB_meal
    df_train['is_FB_meal'] = np.where(df_train['meal'] == 'FB', 1, 0)
    df_test['is_FB_meal'] = np.where(df_test['meal'] == 'FB', 1, 0)

    # market_rist_level -> 인코딩 필요(라이트gbm, 캣부스트 제외)
        # 리스크 레벨 매핑 딕셔너리
    risk_mapping = {
    'Groups': 'High risk',
    'Online TA': 'High risk', 
    'Offline TA/TO': 'Medium risk',
    'Direct': 'Low risk',
    'Corporate': 'Low risk',
    'Complementary': 'Low risk'}
    df_train['market_risk_level'] = df_train['market_segment'].map(risk_mapping)
    df_test['market_risk_level'] = df_test['market_segment'].map(risk_mapping)

    # is_HighRisk_markket_risk
    df_train['is_High_risk_market_risk'] = (df_train['market_risk_level'] == 'High risk').astype(int)
    df_test['is_High_risk_market_risk'] = (df_test['market_risk_level'] == 'High risk').astype(int)
    
    # adr_processed
    # 1. IQR을 사용하여 훈련 데이터(X_tr)의 이상치 범위 계산
    Q1 = df_train['adr'].quantile(0.25)
    Q3 = df_train['adr'].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    lower_bound = Q1 - 1.5 * IQR
    # 2. 이상치(outliers)를 제외한 훈련 데이터의 adr 중앙값 재계산
    # adr은 보통 0보다 크므로 하한선을 0으로 설정하거나 IQR로 계산된 값을 사용
    # 여기서는 IQR로 계산된 값을 사용하여 더 일반적인 방법으로 처리
    adr_filtered_median = df_train.loc[(df_train['adr'] >= lower_bound) & (df_train['adr'] <= upper_bound), 'adr'].median()
    # 3. 훈련 세트(X_tr)에 새로운 'adr_processed' 피처 생성
    # 이상치 범위(lower_bound ~ upper_bound)를 벗어나는 값들을 필터링된 중앙값으로 대체
    df_train['adr_processed'] = np.where(
        (df_train['adr'] < lower_bound) | (df_train['adr'] > upper_bound),
        adr_filtered_median,
        df_train['adr'])
    # 4. 테스트 세트(X_te)에 새로운 'adr_processed' 피처 생성
    # 훈련 데이터에서 계산한 동일한 이상치 범위와 중앙값을 사용
    df_test['adr_processed'] = np.where(
        (df_test['adr'] < lower_bound) | (df_test['adr'] > upper_bound),
        adr_filtered_median,
        df_test['adr'])

    # lead_time_processed
    # 리드타임 구간별 범주화 (0-30, 30-60, 60-90, 90-180, 180-365, 365+)
    # NaN 값을 0으로 채우고 정수로 변환
    df_train['lead_time_processed'] = pd.cut(df_train['lead_time'], 
                                            bins=[0, 30, 60, 90, 180, 365, float('inf')], 
                                            labels=[0, 1, 2, 3, 4, 5]).fillna(0).astype(int)

    df_test['lead_time_processed'] = pd.cut(df_test['lead_time'], 
                                            bins=[0, 30, 60, 90, 180, 365, float('inf')], 
                                            labels=[0, 1, 2, 3, 4, 5]).fillna(0).astype(int)

    # lead_time
    # 2. # 1단계: 훈련 데이터(X_tr)에서 IQR을 사용하여 이상치 범위 계산
    Q1 = df_train['lead_time'].quantile(0.25)
    Q3 = df_train['lead_time'].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    # 2단계: 이상치를 제외한 데이터로 중앙값 재계산
    # lead_time은 음수일 수 없으므로 하한선은 0으로 설정합니다.
    lead_time_filtered_median = df_train.loc[(df_train['lead_time'] >= 0) & (df_train['lead_time'] <= upper_bound), 'lead_time'].median()
    # 3단계: X_tr에 'lead_time_processed' 피처 생성
    # 0 미만이거나 상한선(upper_bound)을 초과하는 값을 필터링된 중앙값으로 대체
    df_train['lead_time_processed'] = np.where(
        (df_train['lead_time'] < 0) | (df_train['lead_time'] > upper_bound),
        lead_time_filtered_median,
        df_train['lead_time'])
    # 4단계: X_te에 'lead_time_processed' 피처 생성
    # 훈련 데이터에서 계산한 동일한 상한선과 중앙값을 사용
    df_test['lead_time_processed'] = np.where(
        (df_test['lead_time'] < 0) | (df_test['lead_time'] > upper_bound),
        lead_time_filtered_median,
        df_test['lead_time'])


    # is_alone
    ## 준호님꺼
    df_train['total_guests'] = df_train['adults'] + df_train['children'] + df_train['babies']
    df_test['total_guests'] = df_test['adults'] + df_test['children'] + df_test['babies']
    # Create 'is_alone' feature for both sets
    # 1 if total_guests is 1, otherwise 0
    df_train['is_alone'] = df_train['total_guests'].apply(lambda x: 1 if x == 1 else 0)
    df_test['is_alone'] = df_test['total_guests'].apply(lambda x: 1 if x == 1 else 0)
    # Optionally, you can drop the intermediate 'total_guests' feature
    df_train = df_train.drop('total_guests', axis=1)
    df_test = df_test.drop('total_guests', axis=1)
    
    # is_resort
    # City Hotel은 0, Resort Hotel은 1로 변환
    df_train['is_resort'] = df_train['hotel'].map({'City Hotel': 0, 'Resort Hotel': 1})
    df_test['is_resort'] = df_test['hotel'].map({'City Hotel': 0, 'Resort Hotel': 1})

    # is_transient
    df_train['is_transient'] = (df_train['customer_type'] == 'Transient').astype(int)
    df_test['is_transient'] = (df_test['customer_type'] == 'Transient').astype(int)

    # total_stays
    df_train['total_stays'] = df_train['stays_in_weekend_nights'] + df_train['stays_in_week_nights']
    df_test['total_stays'] = df_test['stays_in_weekend_nights'] + df_test['stays_in_week_nights']


    return df_train, df_test


In [5]:
x_tr, x_te = do_feature_extraction(x_tr, x_te)

In [13]:
x_tr.shape, y_tr.shape, x_te.shape, y_te.shape

((31331, 43), (31331,), (8669, 43), (8669,))

# 오류 발생하는 이유 찾기
- preprocessing 단계에서 인코딩 전까지 실행
- 인코딩 전 x_ 데이터의 타입이 object 인 컬럼 확인하기
: 
- 사전에 정한 인코딩 대상 컬럼
 : 'hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'customer_type'
- 놓친 인코딩 대상 컬럼 : 'market_risk_level'


In [ ]:
#  preprocessing 단계에서 인코딩 전까지 실행 cleaning 단계

import pandas as pd
import numpy as np


# 결측치 치환
def __fillna(df_train:pd.DataFrame, df_test:pd.DataFrame):
    # train 데이터를 기준으로 채울 값을 정하기
    # df_train.isnull() -> True/False,2차원
    # df_train.isnull().sum() -> 컬럼별 결측치의 갯수, 1차원  ## 통계함수가 들어가면 차원이 줄어듦.
    # -> index는 컬럼임 / value는 결측치의 갯수
    # df_train.isnull().sum()[df_train.isnull().sum() > 0] -> 결측치가 있는 데이터만 조회 -> 1차원 데이터(index=컬럼, value=결측치의 갯수)
    # .index -> index(=컬럼)만 조회
    train_none_cols = df_train.isnull().sum()[df_train.isnull().sum() > 0].index
    test_none_cols = df_test.isnull().sum()[df_test.isnull().sum() > 0].index
    none_cols = list(set(train_none_cols) | set(test_none_cols))  # 짝대기 | : or 의미함 -> 합집합
    # 위의 정한 값을 기준으로 train, test 데이터의 결측치 치환

    for col in none_cols:
        try: 
            _value = df_train[col].mean() # 숫자형 데이터의 경우 평균값 넣기
        except:
            _value = df_test[col].mode()[0] # 범주형 데이터의 경우 최빈값 넣기
        finally:
            # 결측치에 통계값 넣기
            df_train[col] = df_train[col].fillna(_value)
            df_test[col] = df_test[col].fillna(_value)

    return df_train, df_test

# 필요 없는 컬럼 제거 > 주피터파일에서 eda 확인후 결정
def __dropcols(df_train:pd.DataFrame, df_test:pd.DataFrame, drop_cols:list):
    return df_train.drop(drop_cols, axis=1), df_test.drop(drop_cols, axis=1)


# 왜도 / 첨도 처리 : log 변환 > 주피터파일에서 eda 확인후 결정
def __transform_cols(df_train:pd.DataFrame, df_test:pd.DataFrame, transform_cols:list):
    
    for col in transform_cols: # age, fare
        df_train[col] = df_train[col].map(lambda x : np.log1p(x)) 
        df_test[col] = df_test[col].map(lambda x : np.log1p(x))
        ## map은 1차원 데이터(col)만 받아줌. apply는 2차원 데이터(df)도 받아줌.
        ## lambda x : np.log1p(x) -> x(파라미터)를 np.log1p(x)로 변환하고 x로 반환. 그래서 반환값이 변환된 x(리턴)가 됨.

    return df_train, df_test



# df_test -> 제출용 데이터. 데이터가 줄어들면  // transform 안함함
def do_cleansing(df_train:pd.DataFrame, df_test:pd.DataFrame, drop_cols:list):   
    # 1. row 중복 제거
    #df_train = df_train.drop_duplicates()
    # drop_duplicates() -> 중복된거 알아서 없애줌. 받을 인자 없음


    # 2. 결측치 치환(train데이터만 가지고 train, test 데이터의 결측치 치환)
    df_train, df_test = __fillna(df_train, df_test)

    # 3. 필요 없는 컬럼 제거                                      drop_cols를 외부에서 받아옴
    df_train, df_test = __dropcols(df_train, df_test, drop_cols=drop_cols)


    # 4. 왜도 / 첨도 처리
    # df_train, df_test = __transform_cols(df_train, df_test, transform_cols=transform_cols)

    # 5. 검증
    ## train, test 데이터의 컬럼 갯수가 같은지 확인
    assert df_train.shape[1] == df_test.shape[1], "학습용과 테스트용 데이터의 컬럼 갯수가 같지 않습니다."


    return df_train, df_test


drop_cols = ['deposit_type', 'company' ,'agent','reservation_status', 'reservation_status_date', 'assigned_room_type', 'children', 'babies', 'arrival_date_full']
# transform_cols = ['adr', 'lead_time', 'total_stays']

x_tr, x_te = do_cleansing(df_train=x_tr, df_test=x_te, drop_cols=drop_cols)

In [9]:
# 인코딩 단계 타입이 object인 컬럼명 확인
x_tr.select_dtypes(include=['object']).columns.tolist()

['hotel',
 'arrival_date_month',
 'meal',
 'country',
 'market_segment',
 'distribution_channel',
 'reserved_room_type',
 'customer_type',
 'market_risk_level']

# 인코딩까지 해서 전처리 완료하기

In [ ]:
import pandas as pd
import argparse



def do_encoding(df_train:pd.DataFrame, df_test:pd.DataFrame, encoding_cols:list, args):
    """
    모델 타입에 따라 다른 인코딩 적용
    
    Args:
        model_type: 'lightgbm', 'catboost', 'xgboost' 중 선택
    """
    
    if args.model_name in ['lightgbm', 'catboost']:
        # LightGBM, CatBoost: category 타입으로만 변환
        for col in encoding_cols:
            df_train[col] = df_train[col].astype('category')
            df_test[col] = df_test[col].astype('category')
    
    elif args.model_name == 'xgboost':
        # XGBoost: 원-핫 인코딩 필요
        import category_encoders as ce
        encoder = ce.OneHotEncoder(cols=encoding_cols, use_cat_names=True)
        df_train = encoder.fit_transform(df_train)
        df_test = encoder.transform(df_test)
    
    return df_train, df_test 

parser = argparse.ArgumentParser()
parser.add_argument("--model_name", default="lightgbm", type=str)
encoding_cols = ['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'customer_type', 'market_risk_level']
args = parser.parse_args([]) 

### 주피터에서는 args는 결과물을 저장한 변수일 뿐이고 parser를 써야 함!!
#  parser.add_argument(~~)

do_encoding(df_train=x_tr, df_test=x_te, encoding_cols=encoding_cols, args=args)

(              hotel  lead_time  arrival_date_year arrival_date_month  \
 0      Resort Hotel        342               2015               July   
 1      Resort Hotel        737               2015               July   
 2      Resort Hotel          7               2015               July   
 3      Resort Hotel         13               2015               July   
 4      Resort Hotel         14               2015               July   
 ...             ...        ...                ...                ...   
 31326  Resort Hotel         72               2017              March   
 31327  Resort Hotel         30               2017              March   
 31328  Resort Hotel         40               2017              March   
 31329  Resort Hotel          0               2017              March   
 31330  Resort Hotel         80               2017              March   
 
        arrival_date_week_number  arrival_date_day_of_month  \
 0                            27                          1